In [18]:
import numpy as np
import torch
import torchvision.transforms as transforms
from torchvision.datasets import DatasetFolder
from PIL import Image
from torch.utils.data import ConcatDataset,DataLoader,Subset,Dataset
import gc

#数据增强
train_tfm = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(180),
    transforms.RandomGrayscale(),
    transforms.ToTensor(),
    
])

test_tfm = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

batch_size = 128

def load_pic(x):
    return Image.open(x)

train_set_1 = DatasetFolder("/kaggle/input/dataset/food-11/training/labeled", lambda x: Image.open(x), extensions="jpg", transform=train_tfm)
train_set_2 = DatasetFolder("/kaggle/input/dataset/food-11/training/labeled", lambda x: Image.open(x), extensions="jpg", transform=test_tfm)
valid_set = DatasetFolder("/kaggle/input/dataset/food-11/validation", lambda x: Image.open(x), extensions="jpg", transform=test_tfm)
unlabeled_set = DatasetFolder("/kaggle/input/dataset/food-11/training/unlabeled", lambda x: Image.open(x), extensions="jpg", transform=train_tfm)

test_set = DatasetFolder("/kaggle/input/dataset/food-11/testing", lambda x: Image.open(x), extensions="jpg", transform=test_tfm)
train_set = ConcatDataset([train_set_1, train_set_2]) 
print(len(train_set))
# Construct data loaders.
train_loader = DataLoader(train_set, batch_size=batch_size,  num_workers=8, pin_memory=True,shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size,  num_workers=8, pin_memory=True,shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

6160


In [23]:
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.nn as nn

class Mydataset(Dataset):
    def __init__(self, x ,y):
        self.data=x
        self.label=y
    def __getitem__(self,idx):
        return self.data[idx],self.label[idx]
    def __len__(self):
        return len(self.data)
#半监督学习打伪标签
def pseudo_label(model,unlabel_set):
    model.eval()
    print("k")
    with torch.no_grad():
        data_loader=DataLoader(unlabel_set,batch_size=128,shuffle=False)
    for j,data in enumerate(data_loader):
        inputs,_=data
        inputs=inputs.to("cuda")
        outputs=model(inputs)
        softmax=nn.Softmax(dim=-1)
        labels=softmax(outputs)
        _,labels=torch.max(labels,dim=1)
      
        if j==0:
            dataset=Mydataset(x=inputs,y=labels)
        else:
            dataset_2=Mydataset(inputs,labels)
            dataset=ConcatDataset([dataset,dataset_2])
        gc.collect()
        torch.cuda.empty_cache()
    return dataset


#网络结构
class Resblk1(nn.Module):
    def __init__(self,ch_in,ch_out,k,s1,s2,p):
        super(Resblk1,self).__init__()
        self.conv0=nn.Conv2d(ch_in,ch_out,1,2,0)
        self.conv1=nn.Conv2d(ch_in,ch_out,k,s1,p)
        self.conv2=nn.Conv2d(ch_out,ch_out,k,s2,p)
        self.relu=nn.ReLU()
        self.bn=nn.BatchNorm2d(ch_out)

    def forward(self,x):
        x1=self.conv1(x)
        x1=self.bn(x1)
        x=self.conv0(x)
        x=self.bn(x)
        x1=self.relu(x1)
        x1=self.conv2(x1)
        x1=self.bn(x1)
        x1=x1+x
        x1=self.relu(x1)
        return x1

class Resblk2(nn.Module):
    def __init__(self,ch_in,ch_out,k,s1,s2,p):
        super(Resblk2,self).__init__()
        
        self.conv1=nn.Conv2d(ch_in,ch_out,k,s1,p)
        self.conv2=nn.Conv2d(ch_out,ch_out,k,s2,p)
        self.relu=nn.ReLU()
        self.bn=nn.BatchNorm2d(ch_out)

    def forward(self,x):
        x1=self.conv1(x)
        x1=self.bn(x1)
        x1=self.relu(x1)
        x1=self.conv2(x1)
        x1=self.bn(x1)
        x1=x1+x
        x1=self.relu(x1)
        return x1        

        
class Resnet(nn.Module):
    def __init__(self):
        super(Resnet,self).__init__()
        self.conv=nn.Sequential(
        nn.Conv2d(3,64,7,2,3),
        nn.BatchNorm2d(64),
        nn.MaxPool2d(3,2,1),
        Resblk2(64,64,3,1,1,1),
        Resblk2(64,64,3,1,1,1),
        #Resblk1(64,128,3,2,1,1),
        #Resblk2(128,128,3,1,1,1),
        #Resblk1(128,256,3,2,1,1),
        #Resblk2(256,256,3,1,1,1),
        #Resblk1(256,512,3,2,1,1),
        #Resblk2(512,512,3,1,1,1),
        nn.AvgPool2d(7,stride=1)
        )
        self.fc=nn.Sequential(nn.Linear(26*26*64,11) 
                             )
        
    def forward(self,x):
        x=self.conv(x)
        x=x.flatten(1)
        x=self.fc(x)
        
        return x

LEARNING_RATE=0.0001
EPOCH=80

model=Resnet().to("cuda")
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=LEARNING_RATE,weight_decay=1e-5)





train_acc_list=[]
val_acc_list=[]
i=0
semi=False
for i in range(EPOCH):
    train_acc=0.0
    train_loss=0.0
    val_acc=0.0
    val_loss=0.0
    
    #网络训练
    model.train()
    for j,data in enumerate(train_loader):
        inputs,labels=data
       
        inputs=inputs.to("cuda")
        labels=labels.to("cuda")
        outputs=model(inputs)
        loss=criterion(outputs,labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss+=loss
        _,pred=torch.max(outputs,dim=1)
        train_acc+=(pred.cpu()==labels.cpu()).sum().item()
    gc.collect()
    torch.cuda.empty_cache()
    
    #半监督学习
    if semi:
        print("s")
        for k,data in enumerate(unlabeled_loader):
            optimizer.zero_grad()
            inputs,labels=data
            inputs=inputs.to("cuda")
            labels=labels.to("cuda")
            outputs=model(inputs)
            loss=criterion(outputs,labels)
            loss.backward()
            optimizer.step()
    gc.collect()
    torch.cuda.empty_cache()

    #验证集
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(valid_loader):
            inputs,labels=data
            inputs=inputs.to("cuda")
            labels=labels.to("cuda")
            outputs=model(inputs)
            loss=criterion(outputs,labels)
            val_loss+=loss
            _,pred=torch.max(outputs,dim=1)
            val_acc+=(pred.cpu()==labels.cpu()).sum().item()
    
    #准确度和损失值    
    train_acc=train_acc/len(train_set)
    val_acc=val_acc/len(valid_set)
    train_loss=train_loss/len(train_loader)
    val_loss=val_loss/len(valid_loader)
    train_acc_list.append(train_acc)
    val_acc_list.append(val_acc)
    print(i," t",train_acc,"v",val_acc)
    
    #打伪标签
    if(val_acc>1 and semi==False ):
        unlabeled_sett=pseudo_label(model,unlabeled_set)
        unlabeled_loader=DataLoader(unlabeled_sett, batch_size=batch_size, num_workers=0, shuffle=False)
        semi=True

plt.figure()
x=range(0,80)
plt.plot(x,train_acc_list, 'r-')
plt.plot(x,val_acc_list,'r-',color='red')
plt.show()

0  t 0.2305194805194805 v 0.15
1  t 0.3728896103896104 v 0.1787878787878788
2  t 0.3961038961038961 v 0.16666666666666666


KeyboardInterrupt: 